<a href="https://colab.research.google.com/github/addvennture/Aconex_NEC_Mail_Processing/blob/main/Aconex_Mail_Data_Processing_rev0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (0) INTRODUCTION

This code takes data from the Aconex_Mail_Data_Query and processes it to pull out commercial emails for projects using the NEC mail configuration. The output is a spreadsheet summary of mail data which is saved to users google drive account. 

# (1) IMPORTING REFERENCED LIBRARIES

In [ ]:
import requests as rq
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import csv
%load_ext google.colab.data_table

# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# (2) CONNECTING TO GOOGLE DRIVE AND IMPORTING DATA FOR PROCESSING

In [ ]:
# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# (3) SETTING & CHECKING THE ACONEX API CONNECTION

In [ ]:
#ENTER ACONEX API CREDENTIALS
#@title ENTER ACONEX API CREDENTIALS
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}
key = "" #@param {type:"string"}
b_url='https://uk1.aconex.co.uk/api/' #base path for all Aconex web services
headers={'Accept': 'application/vnd.aconex.mail.v2+xml','Authorization': 'Bearer {0}'.format(key)}

# (4) DEFINED FUNCTIIONS

In [ ]:
#FUNCTION CAPTURES RELEVANT FIELD DATA FOR IMPLEMENTED CE's
def ice_tots(prj_id):
  ice_tot_cost=pd.to_numeric(grouped.get_group((prj_id,'(ECC) Compensation Event Implementation Instruction'))['Value 2']).sum()
  ice_tot_delay=pd.to_numeric(grouped.get_group((prj_id,'(ECC) Compensation Event Implementation Instruction'))['Value 3']).sum()
  return (prj_id),(ice_tot_cost),(ice_tot_delay)

In [ ]:
#FUNCTION CAPTURES RELEVANT FIELD DATA FOR PAYMENT CERTS
def paycerts_tots(prj_id):
  paycerts_tot=pd.to_numeric(grouped.get_group((prj_id,'(ECC) Payment Certification'))['Value 2']).sum()
  #ice_tot_delay=pd.to_numeric(grouped.get_group((prj_id,'(ECC) Compensation Event Implementation Instruction'))['Value 3']).sum()
  return (prj_id),(paycerts_tot)

# (5) GENERATING PROJECT SUMMARY DATAFRAME

In [ ]:
#RETRIEVING PROJECT LIST FOR USER:
prj_headers={'User':username,'Authorization': 'Bearer {0}'.format(key)}
prj_data=rq.get('https://uk1.aconex.co.uk/api/projects',auth=(username,password),headers=prj_headers)
prj_root=ET.fromstring(prj_data.text) #Bringing data into element tree for processing
#[elem.tag for elem in prj_root.iter()] #Optional step to check data structure
#prj_root.attrib #Check on total number of projects 

In [ ]:
#PROCESSING PROJECT DATA FOR ALL PROJECTS TO POPULATE INITIAL SUMMARY TABLE:
prj_ids=[]
for child in prj_root.iter('ProjectId'):
    prj_ids.append(child.text)

prj_code=[] #generating list of project codes
for child in prj_root.iter('ProjectCode'):
    sew_cer.append(child.text)
    
prj_name=[]
for child in prj_root.iter('ProjectName'):
    prj_name.append(child.text)

prj_val=[]
for child in prj_root.iter('ProjectValue'):
    prj_val.append(child.text)

strt_date=[]
for child in prj_root.iter('StartDate'):
    strt_date.append(child.text)
    
stop_date=[]
for child in prj_root.iter('StopDate'):
    stop_date.append(child.text)

In [ ]:
#Generates dataframe of project data
prjs_df=pd.DataFrame(data=({'Aconex Id':prj_ids,'Project Code':prj_code,'Name':prj_name,'Value £':prj_val,'Start Date':strt_date,'End Date':stop_date}))
#prjs_df

# (6) PROCESSING AND CLEANING PROJECT MAIL DATA

In [ ]:
#@title SET FILE PATH TO IMPORT MAIL DATA FROM { display-mode: "form" }
file_path = "" #@param {type:"string"}
#IMPORTING CSV FILE AND CREATING A PROJECT MAIL DATAFRAME
raw_df=pd.read_csv(file_path,index_col='Mail Type',names=['Num','Aconex Id','Mail Id','Mail Type','Sent Date','Status','Subject','Field 1','Value 1','Field 2','Value 2','Field 3','Value 3','Field 4','Value 4','Field 5','Value 5','Field 6','Value 6','Field 7','Value 7','Field 8','Value 8','Field 9','Value 9','Field 10','Value 10','Field 11','Value 11'])
#raw_df

In [ ]:
md_df=(raw_df.drop(columns='Num',index=['Workflow Transmittal','Transmittal','Response to RFI','General Correspondence','Technical Query','Advice','New Project Request','Response to Technical Query','Workflow Transmittal','Action Response','Action Item','Request For Information','Supplier Document Transmittal'],errors='ignore')
            .set_index('Aconex Id',append=True).drop_duplicates()) #cleaning dataframe
#md_df

In [ ]:
#SUMMARISING PROJECT MAIL DATA FOR EACH PROJECT
grouped=md_df.groupby(['Aconex Id','Mail Type']) #grouping data by project and mail type
mail_smry=grouped.count().drop(labels=['Sent Date','Subject','Field 1','Value 1','Field 2','Value 2','Field 3','Value 3','Field 4','Value 4','Field 5','Value 5','Field 6','Value 6','Field 7','Value 7','Field 8','Value 8','Field 9','Value 9','Field 10','Value 10','Field 11','Value 11'],axis=1)
mail_smry['Outstanding']=mail_smry['Mail Id']-mail_smry['Status'] #computes outstanding mails and adds column to dataframe
#mail_smry

# (7) ADDING SUMMARY STATISTICS TO PROJECT SUMMARY DATAFRAME

In [ ]:
#Getting implemented CE and Payment Cert data and creating individual dataframes for each
ice_data=[] 
for prj_id in prj_ids:
  try:
    ice_data.append(ice_tots(str(prj_id)))
  except:
    ice_data.append([prj_id,0,0])
ice_df=pd.DataFrame(ice_data,columns=['Aconex Id','ICEs Value (£)','ICEs Delay (days)'])
#ice_df

paycert_data=[]
for prj_id in prj_ids:
  try:
    paycert_data.append(paycerts_tots(str(prj_id)))
  except:
    paycert_data.append([prj_id,0])
paycert_df=pd.DataFrame(paycert_data,columns=['Aconex Id','Certified (£)'])
#paycert_data

combined_df=ice_df.merge(paycert_df,how='outer',on='Aconex Id') #Combining implemented CE and Payment Cert. dataframes
#combined_df

In [ ]:
#MERGING PROJECT SUMMARY DATAFRAME WITH ICE AND PAY CERT DATAFRAME TO GENERATE PROJECT SUMMAY OUTPUT
prjs_smry=prjs_df.merge(combined_df,how='outer',on='Aconex Id')
#prjs_smry

# (8) OUTPUTTING DATAFRAMES TO MS EXCEL AND SAVING TO GOOGLE DRIVE


In [ ]:
with pd.ExcelWriter('report.xlsx') as writer:
  prjs_smry.to_excel(writer,sheet_name='Prj Smry')
  mail_smry.to_excel(writer,sheet_name='Mail Smry')
  md_df.to_excel(writer,sheet_name='Mail Data')

In [ ]:
#@title SET OUTPUT FILE NAME FOR GOOGLE DRIVE
rprt_date = "2021-01-31" #@param {type:"date"}
#Option to permanently save temporary csv file to Google Drive.
#Note requires authentication in Section 3.

ftd=drive.CreateFile({'title':'Aconex_Smry_{0}'.format(rprt_date)}) #Creates file with a name matching the report date
ftd.SetContentFile('/content/report.xlsx') #sets processed mail data from above cells as content to upload to new file
ftd.Upload() #uploads content 
print('title:%s, id:%s' % (ftd['title'],ftd['id'])) #prints file name and google drive id

title:Aconex_Smry_2021-01-31, id:1VMHPSoeV5G6I-4NxngsmQ8KhWqLQQClZ
